In [ ]:
import os
import json
import transformers
from openai import OpenAI

In [ ]:
import os
os.environ['HF_TOKEN'] = "  "

In [ ]:
# =========================
# Config
# =========================
MODEL_ID = "deepseek-ai/DeepSeek-V3.2"
JSON_PATH = "llm_input.jsonl"
TARGET_MATERIAL = "SXL02"

# Router (OpenAI-compatible) generation config
GEN_CONFIG = dict(
    max_tokens=700,      # instead of transformers max_new_tokens
    temperature=0.3,
    top_p=0.9,
)


# =========================
# 1) Load JSONL + filter to SXL02
# =========================
def load_jsonl(path: str):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            try:
                rows.append(json.loads(line))
            except json.JSONDecodeError as e:
                raise ValueError(f"JSONL parse failed: line {i} -> {e}\nProblem line: {line[:200]}") from e
    return rows

def get_material(r):
    # 1) Material at top-level
    for k in ["Material", "MATERIAL", "material", "MATERIAL_KEY"]:
        if k in r:
            return str(r.get(k, "")).strip()

    # 2) Material inside id
    if isinstance(r.get("id"), dict):
        for k in ["Material", "MATERIAL", "material", "MATERIAL_KEY"]:
            if k in r["id"]:
                return str(r["id"].get(k, "")).strip()

    return ""

rows = load_jsonl(JSON_PATH)

target_norm = TARGET_MATERIAL.strip().upper()
payload = [r for r in rows if get_material(r).strip().upper() == target_norm]

if not payload:
    raise ValueError(f"Material='{TARGET_MATERIAL}' not found in JSONL. Check keys/values.")

payload_text = json.dumps(payload, ensure_ascii=False, indent=2)


# =========================
# 2) Build prompt
# =========================
prompt = f"""
You are a B2B demand planning and supply chain analytics expert.
Task: Diagnose the business situation for Material={TARGET_MATERIAL} using ONLY the JSON below.
You MUST analyze relationships between ranking, instock, and POS/USW.

Definitions (use as signals):
- Rank: weekly competitiveness signal (assume lower number = better unless evidence suggests otherwise; if unclear, state assumption)
- Instock: weekly availability signal
- POS / USW: demand signal (use multiple weeks, ignore 1-week noise)

Data rules:
- POS sales: weekly
- Rank and Instock: weekly signals
- ASHIP/ESHIP/Forecast: monthly signals
- POS volatility is normal; infer trend from multiple weeks, not a single point

Decision rules (use as heuristics; apply when relevant; NOT exhaustive):
- A–E are examples. Do not force-fit every case into A–E.
- If patterns don’t match, propose new rule(s) and label them F/G/...
- If signals vary by week, split into phases and diagnose per phase.
- You may conclude "mixed signals" with multiple contributing causes if supported.
- First, infer whether a smaller rank number means better rank from the data; if unclear, state the assumption explicitly.

A) If rank worsens AND instock improves AND USW/POS declines:
   -> demand softness / market demand drop likely (not an availability issue)
B) If rank worsens AND instock declines AND USW/POS is stable or rising:
   -> availability/in-stock issue likely (lost sales due to OOS)
C) If rank improves AND instock declines AND USW/POS rises:
   -> demand is strong; risk of stockout; consider increasing forecast/shipments (upside capture)
D) If rank improves AND instock improves AND USW/POS rises:
   -> strong momentum; validate sustainability; consider forecast uplift + monitor constraints
E) If rank changes but USW/POS is flat:
   -> ranking change may be competitive noise; investigate pricing, promotion, category shifts

Output format (must follow):
1) Observed patterns (rank + instock + POS/USW over time; call out direction clearly)
2) Situation diagnosis (identify primary + secondary drivers; can be 1-3 items)
3) Root causes (ranked 3-5) + Supporting evidence (exact JSON fields used)
4) Action plan:
   - Defensive actions (fix availability, replenishment, ASN, etc.)
   - Offensive actions (forecast uplift, expedite, promo support, etc.)
   - What to check next (specific data you need)
JSON:
{payload_text}
""".strip()


# =========================
# 3) Call HF Router
# =========================
hf_token = os.environ.get("HF_TOKEN")  # recommend environment variable
if not hf_token:
    # If you want to set it directly in code, do it here:
    # hf_token = "hf_..."
    raise EnvironmentError("HF_TOKEN environment variable is missing. Set it first.")

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=hf_token,
)

resp = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {"role": "system", "content": "You are precise and evidence-driven. Follow the output format exactly."},
        {"role": "user", "content": prompt},
    ],
    **GEN_CONFIG,
)

print(resp.choices[0].message.content)
